In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


sns.set()

#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
# from IPython.core.display import display, HTML
from IPython.display import display, HTML

display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
df=pd.read_csv("./datasets/train1.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 12 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   Unnamed: 0.1              1000000 non-null  int64  
 1   Unnamed: 0                1000000 non-null  int64  
 2   id                        1000000 non-null  object 
 3   month                     1000000 non-null  int64  
 4   day_of_month              1000000 non-null  int64  
 5   estimated_departure_time  1000000 non-null  float64
 6   estimated_arrival_time    1000000 non-null  float64
 7   origin_airport_id         1000000 non-null  int64  
 8   destination_airport_id    1000000 non-null  int64  
 9   distance                  1000000 non-null  float64
 10  tail_number               1000000 non-null  object 
 11  delay                     255001 non-null   object 
dtypes: float64(3), int64(6), object(3)
memory usage: 91.6+ MB


In [4]:
df['delay']=df['delay'].fillna('Not_Delayed')

In [5]:
df=df.drop(['id','Unnamed: 0.1','Unnamed: 0'],axis=1)

In [6]:
# from sklearn.preprocessing import LabelEncoder

# labellist= ['Origin_Airport', 'Origin_State', 'Destination_Airport',
#        'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Tail_Number',
#        'Delay']


# for i in labellist:  
#     le = LabelEncoder()  
#     le.fit(df_full['i']) 
#     df_full['i']=le.transform(df_full['i'])

In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()  
 

le.fit(df['tail_number']) 
df['tail_number']=le.transform(df['tail_number'])


le.fit(df['delay']) 
df['delay']=le.transform(df['delay'])


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(df, df['delay'], test_size=0.2, random_state=0)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

stratify=df['delay']

X_train=X_train.drop(['delay'],axis=1)
X_test=X_test.drop(['delay'],axis=1)

(800000, 9) (200000, 9) (800000,) (200000,)


In [9]:
df.isna().sum()

month                       0
day_of_month                0
estimated_departure_time    0
estimated_arrival_time      0
origin_airport_id           0
destination_airport_id      0
distance                    0
tail_number                 0
delay                       0
dtype: int64

In [10]:
X_train, X_valid, y_train, y_valid= train_test_split(X_train,y_train, test_size=0.2, random_state=42,stratify=y_train)

In [11]:
print(X_train.shape, X_valid.shape, X_test.shape)

(640000, 8) (160000, 8) (200000, 8)


In [12]:
from sklearn.linear_model import LogisticRegression
model1=LogisticRegression()
model1.fit(X_train,y_train)
pred1=pd.DataFrame(model1.predict_proba(X_valid))
print(pred1)

               0         1
0       0.057569  0.942431
1       0.036109  0.963891
2       0.047906  0.952094
3       0.036032  0.963968
4       0.031972  0.968028
...          ...       ...
159995  0.028958  0.971042
159996  0.056395  0.943605
159997  0.028274  0.971726
159998  0.048554  0.951446
159999  0.037705  0.962295

[160000 rows x 2 columns]


In [13]:
from sklearn.ensemble import RandomForestClassifier
model2=RandomForestClassifier()
model2.fit(X_train,y_train)
pred2=pd.DataFrame(model2.predict_proba(X_valid))
print(pred2)

               0         1
0       0.057569  0.942431
1       0.036109  0.963891
2       0.047906  0.952094
3       0.036032  0.963968
4       0.031972  0.968028
...          ...       ...
159995  0.028958  0.971042
159996  0.056395  0.943605
159997  0.028274  0.971726
159998  0.048554  0.951446
159999  0.037705  0.962295

[160000 rows x 2 columns]


In [14]:
from sklearn.ensemble import VotingClassifier
model3=VotingClassifier(estimators=[('logistic',model1),('random',model2)],voting='soft')
model3.fit(X_train,y_train)
pred3=pd.DataFrame(model3.predict_proba(X_valid))
print(pred3)

               0         1
0       0.043784  0.956216
1       0.018054  0.981946
2       0.033953  0.966047
3       0.043016  0.956984
4       0.025986  0.974014
...          ...       ...
159995  0.034479  0.965521
159996  0.038197  0.961803
159997  0.039137  0.960863
159998  0.039277  0.960723
159999  0.033853  0.966147

[160000 rows x 2 columns]


In [15]:
# from sklearn.metrics import roc_auc_score
# print('logistic',roc_auc_score(y_valid,pred1.iloc[:,1]))
# print('RandomForest',roc_auc_score(y_valid,pred2.iloc[:,1]))
# print('Voting',roc_auc_score(y_valid,pred3.iloc[:,1])) 

In [16]:
# from sklearn.model_selection import GridSearchCV
# parameters={'n_estimators':[50,100],'max_depth':[4,6]}
# model5=RandomForestClassifier()
# clf=GridSearchCV(estimator=model5, param_grid=parameters, cv=3)
# clf.fit(X_train, y_train)
# print('최적의 파라미터:',clf.best_params_)

In [ ]:
sample_submission = pd.read_csv('./datasets/open/sample_submission.csv', index_col = 0)

In [24]:
pred3 = pred3.rename(columns = {0:'Not_Delayed',1:'Delayed'})
pred3.head(2)

,Not_Delayed,Delayed
0,0.043784,0.956216
1,0.018054,0.981946


In [30]:
sample_submission = pd.read_csv('./datasets/open/sample_submission.csv', index_col = 0)

submission = pd.DataFrame(data=pred3, index=sample_submission.index)
submission.to_csv('./datasets/submit4.csv', index=True)

In [ ]:
# from sklearn.metrics import log_loss

# binary_logloss1 = log_loss(y_valid, pred1)
# print(binary_logloss)